In [1]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/BM-pt3/BM-A-pt3

In [2]:
# import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers.trainer_utils import get_last_checkpoint

# latest = get_last_checkpoint("fpf-7b-instructions-16k-call-v2")

/home/cybertron/anaconda3/envs/sgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
#mesolitica/malaysian-llama2-7b-32k-instructions-v2
tokenizer = AutoTokenizer.from_pretrained('mesolitica/mallam-5B-4096')
# model = AutoModelForCausalLM.from_pretrained(
#     'fpf-7b-instructions-16k-call-v2/checkpoint-9300', 
#     use_flash_attention_2 = True, 
#     torch_dtype = torch.float16,
#     device_map="cuda:0"
# )

model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/mallam-5B-4096', 
    #use_flash_attention_2 = True, 
    torch_dtype = torch.float16,
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████████████| 3/3 [00:03<00:00,  1.08s/it]


In [5]:
from tqdm import tqdm
import os
import random
import json

In [6]:
data_folder = '/media/cybertron/fa54fcb6-b5e1-492e-978a-6389519c168a/llm-benchmarks/fizik-spm/data/'
with open(data_folder+ 'fizik') as fopen:
    text = fopen.read()
    
questions = []
for t in text.split('no: ')[1:]:
    t = t.strip()
    no = t.split('\n')[0]
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)

In [7]:
arange = set(range(len(questions)))

In [8]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
soalan: {row['soalan']}
jawapan:
    """
    return prompt.strip()

def parse_mistral_chat(messages):

    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = ['<s>']
    for u, a in zip(users, assistants):
        texts.append(f'[INST] {u.strip()} [/INST]{a.strip()}</s> ')

    texts.append(f'[INST] {user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [9]:
for i in tqdm(range(len(questions))):
    
    shots = random.sample(arange - {i}, 3)
    prompts = []
    for no, s in enumerate(shots):
        prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    messages = [
        {'role': 'user', 'content': prompt}
    ]
    prompt = parse_mistral_chat(messages)
    inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
    repeat = []
    for _ in range(5):
        try:
            generate_kwargs = dict(
                inputs,
                max_new_tokens=50,
                top_p=0.95,
                top_k=50,
                temperature=0.1,
                do_sample=True,
                num_beams=1,
                repetition_penalty=1.05,
            )
            r = model.generate(**generate_kwargs)
            splitted = tokenizer.decode(r[0]).split('[/INST]')[1].strip().replace('</s>', '').replace('.', '').replace(',', '').strip().split()
            splitted = [t for t in splitted if len(t) == 1 and t in {'A', 'B', 'C', 'D'}]
            repeat.append(splitted[0])
    
        except Exception as e:
            pass
    
    questions[i]['output'] = repeat

  0%|                                                    | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_9916/1633649318.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  shots = random.sample(arange - {i}, 3)
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
  5%|██▏                                         | 1/20 [00:11<03:35, 11.34s/it]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_

In [10]:
def most_common(l):
    return max(set(l), key=l.count)

In [11]:
filtered = [q for q in questions if 'output' in q and len(q['output'])]
correct = 0
for q in filtered:
    correct += most_common(q['output']) == q['jawapan']
(correct / len(filtered)) * 100

25.0